In [1]:
import uproot
import numpy as np
from matplotlib import pyplot as plt

from os import path

In [6]:
#Open double ROOT file 
file_path = "/Users/bertabeltran/mist/data/efficiency_test/full_double_eff_test_spectra_waves/RAW/SDataR_full_double_eff_test_spectra_waves.root"
#file_path ="/Users/bertabeltran/mist/data/efficiency_test/sec5_double_eff_test_spectra_waves/RAW/DataR_sec5_double_eff_test_spectra_waves.root"
file=uproot.open(file_path)
tree=file["Data_R"]
print(tree.keys())

#Load data for channels and timestamps
#timestamps = tree["Timestamp"].array(library = "np")*10**(-3) #Convert picoseconds to nano
timestamps = tree["Timestamp"].array(library = "np") # picoseconds
channel = tree["Channel"].array(library = "np")
energy = tree["Energy"].array(library = "np")
flags = tree["Flags"].array(library = "np")

KeyInFileError: not found: 'Data_R' (with any cycle number)

    Available keys: (none!)

in file /Users/bertabeltran/mist/data/efficiency_test/full_double_eff_test_spectra_waves/RAW/SDataR_full_double_eff_test_spectra_waves.root

In [5]:
#Create masks for channels
CH0_mask = channel == 0
CH1_mask = channel == 1

#Get timestamps for each channel (time in picoseconds from the begining of the run)
CH0_timestamps = timestamps[CH0_mask]
CH1_timestamps = timestamps[CH1_mask]


#Calculate energy of events on each channel
CH0_energy = energy[CH0_mask]
CH1_energy = energy[CH1_mask]

#calcuate the time difference between events in chn 0
CH0_time_diff = np.diff(CH0_timestamps)
CH0_time_diff=CH0_time_diff/1e+12 #make it seconds 

NameError: name 'channel' is not defined

In [ ]:
len(CH0_timestamps)

In [ ]:
len(CH1_timestamps)

In [ ]:
# plot the time differences distribution to see if it looks right 

plt.hist(CH0_time_diff, bins = 100, histtype='stepfilled' ,density=True,color ='orange')
#plt.hist(CH0_time_diff, bins = 100,range=(0,4), histtype='stepfilled' ,density=True,color ='orange')
ax=plt.gca()
ax.set_title("Sec 2 double stack Event Time Differences")
ax.set_xlabel("Time differeces (s) ")
plt.yscale('log')


plt.show()


In [ ]:
# now I am going to try and fit the time differences plot with curve_fit

import numpy as np
from scipy.optimize import curve_fit
#firts I define the exponential fucntion that I will be fitting 
def expo(x, a, b):
    return  np.exp(a+(b*x))

#here I caculate the bin centers for the fit               
def bincenters(bins):
    return (bins[1:] + bins[:-1]) / 2
                   
# define the histogram that I will be fitting to and return the bin contents and the bid edeges    
bin_cont, bin_edges, patches = plt.hist(CH0_time_diff, bins = 100,histtype='stepfilled' , color ='blue',label ="Time Differences")
                   
# here I do the fit a return the fit values     
param, param_cov = curve_fit(expo, bincenters(bin_edges),bin_cont)
                   
# print the values     
print("Expo function coefficients:")
print(param)
print("Covariance of coefficients:")
print(param_cov)

# now I also want to draw the fitted function on top of the histogram 
fit_func= expo(bincenters(bin_edges),param[0] ,param[1] )
plt.plot(bincenters(bin_edges), fit_func, 'red', linewidth=2, label ="fit function ")       

fit_eq= r"exp$({:.2f} {:.2f}x)$".format(param[0] ,param[1])

ax=plt.gca()
ax.text(8, 100, fit_eq, fontsize=15)
ax.text(8, 40, "rate={:.2f} cps".format(abs(param[1])), fontsize=15)

plt.legend()
plt.yscale('log')
ax.set_title("Double stack section 4 time diffeences")
ax.set_xlabel("Time differeces (s) ")
ax.set_ylabel('Number of Events')
plt.show()

In [ ]:
# caculate the event rate for chan 0 by looking at the rate plot 
# first convert the time stamps to seconds 
CH0_timestamps_sec=CH0_timestamps/1e+12
#now I floor this array to see only seconds 
rate=np.floor(CH0_timestamps_sec)
rate

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
bin_contents, bin_edges, patches =plt.hist(rate,bins = 17658, histtype='bar')
#bin_contents, bin_edges, patches =plt.hist(rate, bins = 6832,range=(0,6831))
#bin_centers = bin_edges[:-1] + np.diff(bin_edges) / 2
ax = plt.gca()
ax.set_ylabel("Count per second")
ax.set_xlabel("Seconds from the begining of the run")

In [ ]:
np.mean(bin_contents)

In [ ]:
CH0_timestamps_min=CH0_timestamps_sec/60.
CH0_timestamps_min

In [ ]:
rate_min_0=np.floor(CH0_timestamps_min)
rate_min_0

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
bin_contents, bin_edges, patches =plt.hist(rate_min_0,bins = 294, histtype='step')
#bin_contents, bin_edges, patches =plt.hist(rate, bins = 6832,range=(0,6831))
#bin_centers = bin_edges[:-1] + np.diff(bin_edges) / 2
ax = plt.gca()
ax.set_ylabel("Count per minute")
ax.set_xlabel("Minutes from the begining of the run")

In [ ]:
#Open triple ROOT file 
file_path_3 = "/Users/bertabeltran/mist/data/efficiency_test/full_triple_eff_test_spectra_waves/RAW/SDataR_full_triple_eff_test_spectra_waves.root"
file3=uproot.open(file_path_3)
tree3=file3["Data_R"]
print(tree3.keys())

#Load data for channels and timestamps
#timestamps = tree["Timestamp"].array(library = "np")*10**(-3) #Convert picoseconds to nano
timestamps3 = tree3["Timestamp"].array(library = "np") # picoseconds
channel3 = tree3["Channel"].array(library = "np")
energy3 = tree3["Energy"].array(library = "np")
flags3 = tree3["Flags"].array(library = "np")

In [ ]:
#Create masks for channels
CH2_mask = channel3 == 2


#Get timestamps for each channel (time in picoseconds from the begining of the run)
CH2_timestamps = timestamps3[CH2_mask]


#Calculate energy of events on each channel
CH2_energy = energy3[CH2_mask]


#calcuate the time difference between events in chn 0
CH2_time_diff = np.diff(CH2_timestamps)
CH2_time_diff_sec=CH2_time_diff/1e+12 #make it seconds 
CH2_timestamps

In [ ]:
# plot the time differences distribution to see if it looks right 

plt.hist(CH2_time_diff_sec, bins = 100, histtype='stepfilled',color ='orange')
#plt.hist(CH0_time_diff, bins = 100,range=(0,4), histtype='stepfilled' ,density=True,color ='orange')
ax=plt.gca()
ax.set_title("Scint2  Event Time Differences")
ax.set_xlabel("Time differeces (s) ")
plt.yscale('log')


plt.show()


In [ ]:
# now I am going to try and fit the time differences plot with curve_fit

import numpy as np
from scipy.optimize import curve_fit
#firts I define the exponential fucntion that I will be fitting 
def expo(x, a, b):
    return  np.exp(a+(b*x))

#here I caculate the bin centers for the fit               
def bincenters(bins):
    return (bins[1:] + bins[:-1]) / 2
                   
# define the histogram that I will be fitting to and return the bin contents and the bid edeges    
bin_cont, bin_edges, patches = plt.hist(CH2_time_diff_sec, bins = 100,histtype='stepfilled' , color ='blue',label ="Time Differences")
                   
# here I do the fit a return the fit values     
param, param_cov = curve_fit(expo, bincenters(bin_edges),bin_cont)
                   
# print the values     
print("Expo function coefficients:")
print(param)
print("Covariance of coefficients:")
print(param_cov)

# now I also want to draw the fitted function on top of the histogram 
fit_func= expo(bincenters(bin_edges),param[0] ,param[1] )
plt.plot(bincenters(bin_edges), fit_func, 'red', linewidth=2, label ="fit function ")       

fit_eq= r"exp$({:.2f} {:.2f}x)$".format(param[0] ,param[1])

ax=plt.gca()
ax.text(8, 100, fit_eq, fontsize=15)
ax.text(8, 40, "rate={:.2f} cps".format(abs(param[1])), fontsize=15)

plt.legend()
plt.yscale('log')
ax.set_title("Section 5 test section time diffeences")
ax.set_xlabel("Time differeces (s) ")
ax.set_ylabel('Number of Events')
plt.show()

In [ ]:
CH2_timestamps_sec=CH2_timestamps/1e+12
CH2_timestamps_min=CH2_timestamps_sec/60
rate_min_2=np.floor(CH2_timestamps_min)
rate_min_2
len(rate_min_2)

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
plt.hist(rate_min_0,bins = 297, histtype='step',color='red',label='double stack')
bin_contents, bin_edges, patches =plt.hist(rate_min_2,bins = 297, histtype='step',label='test section')

#bin_contents, bin_edges, patches =plt.hist(rate, bins = 6832,range=(0,6831))
#bin_centers = bin_edges[:-1] + np.diff(bin_edges) / 2
ax = plt.gca()
ax.set_ylabel("Count per minute")
ax.set_xlabel("Minutes from the begining of the run")
plt.title('Section 2 ( not corrected by area differences)')
ax.legend(loc=(0.4, 0.85))


